In [143]:
##
##Script to get List of categories available in Yelp that list restraurants in ranking
##
import pandas as pd
import numpy as np
#for creating geo coordinate combinations
import itertools  as it
from yelpapi import YelpAPI
import requests     # how python goes onto the internet!
import re           # regex
from bs4 import BeautifulSoup # a python HTML parser (version 3)
from urllib2 import Request, urlopen
import json
from pandas.io.json import json_normalize
%matplotlib inline

#### https://www.yelp.com/developers/v2/manage_api_keys refresh to generate new keys

In [144]:
api_key = 'SD5ybcet2QSwWXz2N7M5lw'
api_secret = 'EJBXJBAFgEka0br1ynT6HwWJVT8'
access_token = 'EESADq5kSfBL4XkbxzwJxsqhZIgQHbJs'
access_secret = 'JMU-eh8i0eVUUYxJcjelLkZEfTY'

#### activate yelpapi with personal keys

In [145]:
yelp_api = YelpAPI(api_key, api_secret, access_token, access_secret)

## Create a method to grab yelp business with a keyword in a lat-long bound

In [146]:
#Create method to call yelp api with term and bound filters
def getbiz(term, bounds):
    search_results_df = pd.DataFrame({})
    offset = 0
    while offset<= 980:
        temp_dict = yelp_api.search_query(term= term, bounds=bounds, offset=offset)
        temp_df = pd.DataFrame(temp_dict['businesses'])
        search_results_df = search_results_df.append(temp_df, ignore_index=True)
        if temp_df.shape[0]<20:
            print 'Results exhausted at '+ str(temp_df.shape[0]+offset)+' results at offset '+str(offset)
            break;
        offset += 20
    return search_results_df;

In [147]:

#Specify the keyword to search
term = 'bar'

#Specify the lat lon to specify bounds
nw_latitude=37.810298
nw_longitude=-122.522450
se_latitude=37.704093
se_longitude=-122.375803

#Specify how many rows and columns you wish to devide the geographical bounds into
block = 15

#Find latitude and longitude blocks coordinates
lat = np.arange(se_latitude, nw_latitude, (nw_latitude-se_latitude)/block)
lon = np.arange(se_longitude, nw_longitude, (nw_longitude-se_longitude)/block)

#build a block squared list of coordinates
import itertools
listoflist = lat,lon

#make all combinations of bounds required for calculation
comb  = list(itertools.product(*listoflist))

#sequence the list of coordinates to run north (east to west) to south (east to west)
comb = comb[::-1]

In [148]:
#Function to get 1 if a number is not a multiple of something 
import math
def rem(x,y):
    res = x % y
    return math.copysign(res,x)


In [149]:
#Create a list of all bounds to pass through getbiz() method

i=0
bounds = []
#print (block*(block-1)-1)
while i<=(block*(block-1)-1):
    if rem(i,block) != (block-1):
        bounds = bounds + [str(comb[i][0])+','+str(comb[i][1])+'|'+str(comb[i+1+block][0])+','+str(comb[i+1+block][1])]
        #print len(bounds)
        i += 1
    elif rem(i,block) == (block-1):
        i += 1
        #print 'this is where im at'+str(i)

In [150]:
#Run getbiz method for each element in bounds
b = 0
biz=pd.DataFrame({})
while b <= (len(bounds)-1):
    biz = biz.append(getbiz(term,bounds[b]), ignore_index=True)
    print b
    b+=1
biz.shape

Results exhausted at 0 results at offset 0
0
Results exhausted at 0 results at offset 0
1
Results exhausted at 0 results at offset 0
2
Results exhausted at 1 results at offset 0
3
Results exhausted at 6 results at offset 0
4
Results exhausted at 8 results at offset 0
5
Results exhausted at 5 results at offset 0
6
Results exhausted at 52 results at offset 40
7
Results exhausted at 21 results at offset 20
8
Results exhausted at 24 results at offset 20
9
Results exhausted at 78 results at offset 60
10
Results exhausted at 42 results at offset 40
11
Results exhausted at 1 results at offset 0
12
Results exhausted at 0 results at offset 0
13
Results exhausted at 0 results at offset 0
14
Results exhausted at 0 results at offset 0
15
Results exhausted at 1 results at offset 0
16
Results exhausted at 1 results at offset 0
17
Results exhausted at 2 results at offset 0
18
Results exhausted at 3 results at offset 0
19
Results exhausted at 3 results at offset 0
20
Results exhausted at 3 results at 

(2309, 22)

In [141]:
biz.columns

Index([u'categories', u'deals', u'display_phone', u'gift_certificates', u'id',
       u'image_url', u'is_claimed', u'is_closed', u'location',
       u'menu_date_updated', u'menu_provider', u'mobile_url', u'name',
       u'phone', u'rating', u'rating_img_url', u'rating_img_url_large',
       u'rating_img_url_small', u'review_count', u'snippet_image_url',
       u'snippet_text', u'url'],
      dtype='object')

In [483]:
#Export to csv
biz.to_csv('../data/yelp_east3.csv',encoding='utf-8')